In [1]:
import numpy as np

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset
from l5kit.rasterization import build_rasterizer

import os

In [2]:
os.environ['L5KIT_DATA_FOLDER'] = '../../dataset'
dm = LocalDataManager(None)

cfg = load_config_data('./config.yaml')
print(cfg)

{'format_version': 4, 'model_params': {'model_architecture': 'resnet50', 'history_num_frames': 0, 'history_step_size': 1, 'history_delta_time': 0.1, 'future_num_frames': 50, 'future_step_size': 1, 'future_delta_time': 0.1}, 'raster_params': {'raster_size': [224, 224], 'pixel_size': [0.5, 0.5], 'ego_center': [0.25, 0.5], 'map_type': 'py_semantic', 'satellite_map_key': 'aerial_map/aerial_map.png', 'semantic_map_key': 'semantic_map/semantic_map.pb', 'dataset_meta_key': 'meta.json', 'filter_agents_threshold': 0.5, 'disable_traffic_light_faces': False}, 'train_data_loader': {'key': 'scenes/sample.zarr', 'batch_size': 12, 'shuffle': True, 'num_workers': 16}, 'val_data_loader': {'key': 'scenes/sample.zarr', 'batch_size': 12, 'shuffle': False, 'num_workers': 16}, 'train_params': {'checkpoint_every_n_steps': 10000, 'max_num_steps': 5, 'eval_every_n_steps': 10000}}


In [3]:
train_cfg = cfg['train_data_loader']
rasterizer = build_rasterizer(cfg, dm)
train_zarr = ChunkedDataset(dm.require(train_cfg['key'])).open()

In [22]:
def map_frame_array_to_dict(array):
    return dict(zip(['timestamp', 'agent_index_interval', 'ego_translation', 'ego_rotation'], array))

def map_scene_array_to_dict(array):
    return dict(zip(['frame_index_interval', 'host', 'start_time', 'end_time'], array))

def map_agent_array_to_dict(array):
    return dict(zip(['centroid', 'extent', 'yaw', 'velocity', 'track_id', 'label_probabilities'], array))

In [23]:
scenes = [map_scene_array_to_dict(s) for s in train_zarr.scenes]
frames = [map_frame_array_to_dict(f) for f in train_zarr.frames]
agents = [map_agent_array_to_dict(a) for a in train_zarr.agents]

In [24]:
def distance_from_center(agent1, agent2):
    '''
    Calculates the distance between the centroids of two agents
    @param agent1 - AgentDataset dictionary that represents first agent
    @param agent2 - AgentDataset dictionary that represents second agent
    @return - numpy float distance between the two agents' centroids
    '''
    return np.linalg.norm(agent1['centroid'] - agent2['centroid'])

In [25]:
def distance_from_edge(agent1, agent2):
    '''
    Calculates the distance between the closest edges of two agents
    @param agent1 - AgentDataset dictionary that represents first agent
    @param agent2 - AgentDataset dictionary that represents second agent
    @return - numpy float distance between two agents' closest edges
    '''
        
    # generate orthogonal axes for each agent
    a1_length_axis = agent1['extent'][1] / 2 * np.asarray([np.cos(agent1['yaw']), np.sin(agent1['yaw'])])
    a1_width_axis  = agent1['extent'][0] / 2 * np.asarray([np.sin(agent1['yaw']), np.cos(agent1['yaw'])])
    a2_length_axis = agent2['extent'][1] / 2 * np.asarray([np.cos(agent2['yaw']), np.sin(agent2['yaw'])])
    a2_width_axis  = agent2['extent'][0] / 2 * np.asarray([np.sin(agent2['yaw']), np.cos(agent2['yaw'])])
        
    # find two opposite corners of rectangles
    a1_corners = np.asarray([agent1['centroid'] + a1_length_axis * a + a1_width_axis * a for a in [-1, 1]])
    a2_corners = np.asarray([agent2['centroid'] + a2_length_axis * a + a2_width_axis * a for a in [-1, 1]])
    
    # convert corners into cartesian intervals
    a1_max = np.maximum(*a1_corners)
    a1_min = np.minimum(*a1_corners)
    a2_max = np.maximum(*a2_corners)
    a2_min = np.minimum(*a2_corners)
    
    # compute minkowski distance for rectangles
    y = np.maximum(0, np.maximum(a1_min - a2_max, a2_min - a1_max))
    
    return np.sum(np.abs(y) ** 2, axis=-1) ** (1/2)

In [26]:
distance_from_center(agents[1], agents[5])

53.94792262705793

In [27]:
distance_from_edge(agents[1], agents[5])

51.33004967277152

In [70]:
# playing around with scene 0

scene = scenes[99]
scene

{'frame_index_interval': array([24590, 24838]),
 'host': 'host-a101',
 'start_time': 1571844710505151232,
 'end_time': 1571844735505151232}

In [113]:
# have this be numpy array from start
data = np.array([[0,0,0,0,0]])

fii = scene['frame_index_interval']
for frame in frames[fii[0]: fii[1]]:
    frame_data = []
    
    aii = frame['agent_index_interval']
    for agent in agents[aii[0]: aii[1]]:
        # concatenate these three different features to one embedding
        embedding = np.concatenate((agent['centroid'], agent['velocity'], [agent['yaw']]))
        frame_data.append(embedding)

    data = np.append(data, frame_data, axis=0)
    break

data = data[1:]

In [108]:
# normalize data
def normalize_data():
    pass

In [120]:
from sklearn.cluster import MeanShift, estimate_bandwidth

bandwidth = estimate_bandwidth(data, quantile=0.2, n_samples=len(data))

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(data)

labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print(f"number of clusters {n_clusters_}")

number of clusters 4


In [114]:
data

array([[ 3.80800873e+02, -1.15475685e+02,  0.00000000e+00,
         0.00000000e+00, -8.32659006e-01],
       [ 3.94077271e+02, -1.30089493e+02,  0.00000000e+00,
         0.00000000e+00, -8.22044134e-01],
       [ 3.89383881e+02, -1.24909294e+02,  0.00000000e+00,
         0.00000000e+00, -8.33621264e-01],
       [ 3.85541260e+02, -1.20462067e+02,  0.00000000e+00,
         0.00000000e+00, -8.35427046e-01],
       [ 4.16149689e+02, -1.54887161e+02,  0.00000000e+00,
         0.00000000e+00, -8.08126211e-01],
       [ 4.37675354e+02, -1.30129547e+02,  0.00000000e+00,
         0.00000000e+00, -2.59026027e+00],
       [ 4.13354889e+02, -1.18505775e+02,  0.00000000e+00,
         0.00000000e+00,  2.27939487e+00],
       [ 4.09533905e+02, -1.47839661e+02,  0.00000000e+00,
         0.00000000e+00, -8.51108551e-01],
       [ 4.20132507e+02, -1.59042801e+02,  0.00000000e+00,
         0.00000000e+00, -7.97823906e-01],
       [ 4.00011932e+02, -1.36162827e+02,  0.00000000e+00,
         0.00000000e+00